In [1]:
from sage.rings.finite_rings.hom_finite_field import FiniteFieldHomomorphism_generic

In [2]:
def get_defining_set(n, pol, field):
    #defining_set = [field(0)]
    defining_set = []
    
    while len(defining_set) != n:
        aux = field.random_element()
        if pol(aux) != 0 and aux not in defining_set:
            defining_set = defining_set + [aux]
    
    return defining_set

In [3]:
def random_word(n, field):
    v = []

    while len(v) != n:
        v = v + [choice(field.list())]

    word = vector(field, v)
    
    return word

In [4]:
def random_error(n, num_errors, field):
    v = [0] * n
    i = 0
    
    while i < num_errors:
        m = randint(0, len(v)-1)
        v[m] = choice(field.list())
        i += 1

    e = vector(field, v)
    
    return e

# Goppa code

In [5]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``field`` -- finite field on which `self` is defined.
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite field `\GF{p^m}` extending from `field`.
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are not roots of `generating_pol`
    """
    def __init__(self, defining_set, generating_pol, field):
        """
        Initialize.
        """
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = field
        self._field_L = generating_pol.base_ring()

        if not self._field.is_field() or not self._field.is_finite():
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
        if self.get_dimension() == 0:
            raise ValueError("ERROR. Code dimension is null")
    
    def _repr_(self):
        """
        Representation of a Goppa code.
        """
        return "[{}, {}] Goppa code".format(self._length, self.get_dimension())
    
    def get_generating_pol(self):
        """
        Return the generating polynomial of ``self``.
        """ 
        return self._generating_pol
    
    def get_defining_set(self):
        """
        Return the defining set of ``self``.
        """ 
        return self._defining_set
    
    def get_parity_pol(self):
        """
        Return the parity polynomial of ``self``.
        """
        parity_pol = list()
        
        for elem in self._defining_set:
            parity_pol.append((self._generating_pol.parent().gen() - elem).inverse_mod(self._generating_pol))
        
        return parity_pol
    
    def get_parity_check_matrix(self):
        """
        Return a parity check matrix of ``self``.
        """
        V, from_V, to_V = self._field_L.vector_space(self._field, map = True)

        parity = self.get_parity_pol()

        vector_L = []
        for i in range(len(parity)):
            vector_L = vector_L + parity[i].list()

        vector_F = []
        for i in range(len(vector_L)):
            vector_F = vector_F + to_V(vector_L[i]).list()

        matriz_F = matrix(self._length, vector_F)
        matriz_F = matriz_F.T
        
        return matriz_F
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of ``self``.
        """
        H = self.get_parity_check_matrix()
        G = transpose(H).left_kernel().basis_matrix()
        
        return G
    
    def get_dimension(self):
        """
        Return the dimension of the code.
        """
        
        return rank(self.get_generator_matrix())
    
    

# Goppa encoder

In [6]:
class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self.code().get_generator_matrix()
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m``: a vector to encode
        """
        return m * self.get_generator_matrix()
    
Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder

# Goppa decoder

In [7]:
class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code``: code associated with the decoder
    """
    
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._defining_set = self.code().get_defining_set()
    
    def _repr_(self):
        """
        Representation of a decoder for a Goppa code
        """
        return "Decoder for {}".format(self.code())
    
    def get_syndrome(self, c):
        """
        Return the syndrome polynomial
        
        INPUT:
        - ``c``: a element of the input space of ``self``.
        """
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        
        h = self.code().get_parity_pol()
        
        syndrome = 0
        
        for i in range(len(h)):    
            syndrome = syndrome + embFL(c[i])*h[i]
            
        return syndrome
    
    def get_generating_pol(self):
        """
        Return the generating polynomial
        """
        return self._generating_pol
    
    def decode_to_code(self, word):
        r"""
        Corrects the errors in ``word`` and returns a codeword.
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        i = 1
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        secLF = embFL.section()

        # Step 1
        S = self.get_syndrome(word)
        
        if S == 0:
            return word

        # Step 2
        r_prev = self.get_generating_pol()
        t = floor(self.get_generating_pol().degree()/2)
        r_i = S
        U_prev = 0
        U_i = 1
        
        # Steps 3 and 4
        while r_i.degree() >= t:
            (q, r) = r_prev.quo_rem(r_i)
            aux_r_i = r_i
            aux_U_i = U_i
            r_i = r
            U_i = q * U_i + U_prev
            r_prev = aux_r_i
            U_prev = aux_U_i  
            i += 1
            
        # Step 5
        # make sigma monic     
        sigma = U_i/U_i.coefficients()[-1]
        eta = (-1)^i * r_i / U_i.coefficients()[-1]
        
        # roots of sigma are the locations of the errors
        roots_loc = []
        
        for root in sigma.roots():
            roots_loc = roots_loc + [self._defining_set.index(root[0])]
        
        error = [0] * len(self._defining_set)
        x = self.get_generating_pol().parent().gen()
        sigma_diff = sigma.diff(x)
        
        for i in range(0, len(sigma.roots())):
            error[roots_loc[i]] = secLF(eta.subs(x=sigma.roots()[i][0])/(sigma_diff.subs(x=sigma.roots()[i][0])))
        
        x = word - vector(self.code()._field, error)
        
        return x

    def decode_to_message(self, word):
        word = self.decode_to_code(word)
        return self.code().get_generator_matrix().solve_left(word)
        #return self.code().unencode(self.decode_to_code(word))

Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

# Examples

#### Example 1

In [8]:
F = GF(2^2)
L = GF(2^6)
a = L.gen()
b = F.gen()
R.<x> = L[]
g = x^5 + a*x^2 + 1
n = 30
defining_set = get_defining_set(n, g, L)

In [9]:
# Goppa code
C = Goppa(defining_set, g, F)

print("Goppa code:")
print(C)

print("\nParity matrix:")
H = C.get_parity_check_matrix()
show(H)
#print(latex(H))

print("\nGenerator matrix:")
G = C.get_generator_matrix()
show(G)

Goppa code:
[30, 15] Goppa code

Parity matrix:


[     1      1      0      0      1 z2 + 1      1 z2 + 1      1 z2 + 1      0      0 z2 + 1      1 z2 + 1 z2 + 1      1      0      1      1      1     z2 z2 + 1      1 z2 + 1     z2     z2     z2      1     z2]
[z2 + 1     z2      0      0      1      1      0      1      1      0 z2 + 1      1 z2 + 1 z2 + 1 z2 + 1      0      1      1 z2 + 1      1      0      0     z2      1      0     z2     z2      1     z2 z2 + 1]
[    z2      1     z2      1 z2 + 1 z2 + 1      0      0      1 z2 + 1 z2 + 1      1      0      1      0 z2 + 1 z2 + 1      1      0      0     z2      0      0 z2 + 1 z2 + 1      0 z2 + 1      0      0      1]
[     0     z2      1 z2 + 1     z2     z2     z2      0      1 z2 + 1 z2 + 1      0      0      0      1      0 z2 + 1     z2 z2 + 1 z2 + 1      1 z2 + 1 z2 + 1      1     z2      0 z2 + 1      0 z2 + 1      0]
[     0      0     z2 z2 + 1 z2 + 1 z2 + 1 z2 + 1     z2 z2 + 1      1     z2 z2 + 1 z2 + 1     z2      1 z2 + 1      0     z2     z2     z2     z2 z2 + 1     z2 z2 + 1      0      1 z2 + 1      0     z2 z2 + 1]
[     1     z2     z2      0 z2 + 1     z2     z2 z2 + 1      0 z2 + 1     z2     z2      0     z2      1      0      1 z2 + 1     z2     z2 z2 + 1     z2      1     z2 z2 + 1 z2 + 1      1      1      1      0]
[    z2      0      1 z2 + 1 z2 + 1      0      1 z2 + 1 z2 + 1     z2      0     z2      1     z2      1      0      0     z2 z2 + 1      0 z2 + 1      0 z2 + 1     z2      1      0      0      0 z2 + 1      0]
[    z2      1     z2      1      0      1 z2 + 1      1      0      1 z2 + 1      0      1 z2 + 1      1 z2 + 1 z2 + 1     z2     z2     z2     z2      0 z2 + 1      1      1      0     z2 z2 + 1     z2      1]
[     0     z2 z2 + 1      0      0 z2 + 1 z2 + 1      1      0      0      0 z2 + 1      0 z2 + 1      1 z2 + 1      1     z2      0      1     z2     z2     z2 z2 + 1 z2 + 1      1 z2 + 1      1      0     z2]
[z2 + 1      0     z2     z2      0      0      1      1      0 z2 + 1      1     z2      0      0      0     z2      1     z2     z2     z2      0     z2     z2     z2      1      0      0      1     z2      1]
[z2 + 1      0     z2      1      1 z2 + 1     z2      1     z2 z2 + 1     z2 z2 + 1      1      1     z2 z2 + 1     z2      1      1      1 z2 + 1     z2     z2      1 z2 + 1 z2 + 1 z2 + 1      1 z2 + 1 z2 + 1]
[     1     z2     z2      0      0     z2     z2      0      1      1      1 z2 + 1      0      0 z2 + 1      1      0      1      1     z2      0      0      0      1      0      1      1 z2 + 1      1     z2]
[    z2 z2 + 1     z2 z2 + 1      0      0      0      0 z2 + 1      0     z2      0 z2 + 1 z2 + 1 z2 + 1     z2     z2 z2 + 1 z2 + 1     z2 z2 + 1      0 z2 + 1 z2 + 1     z2     z2      0 z2 + 1      0      1]
[     1 z2 + 1 z2 + 1      0      0     z2      0 z2 + 1      1     z2     z2     z2      1      1      0      1      1      0      0      1 z2 + 1     z2     z2      1     z2      0      1 z2 + 1      0     z2]
[     1 z2 + 1     z2 z2 + 1     z2      1     z2 z2 + 1      1      0 z2 + 1     z2     z2 z2 + 1      1      1      1 z2 + 1      1      0 z2 + 1 z2 + 1      0     z2     z2      1     z2      1     z2      1]


Generator matrix:


[     1      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0 z2 + 1      1      0      0     z2 z2 + 1      0     z2      0      1      0     z2 z2 + 1 z2 + 1]
[     0      1      0      0      0      0      0      0      0      0      0     z2      0      0      0      0     z2      1     z2     z2      1      1     z2 z2 + 1      0      0 z2 + 1      0      1      1]
[     0      0      1      0      0      0      0      0      0      0      0     z2      0      0      0      0 z2 + 1      0      0      1 z2 + 1      0     z2     z2 z2 + 1      1      0      0     z2      0]
[     0      0      0      1      0      0      0      0      0      0      0      0      0      0      0      0      0     z2 z2 + 1      0 z2 + 1 z2 + 1 z2 + 1      1     z2 z2 + 1     z2      0 z2 + 1      1]
[     0      0      0      0      1      0      0      0      0      0      0     z2      0      0      0      0      0      0     z2      0      1      0      1     z2 z2 + 1     z2      1     z2     z2 z2 + 1]
[     0      0      0      0      0      1      0      0      0      0      0      0      0      0      0      0     z2     z2     z2     z2      0      1      1 z2 + 1 z2 + 1 z2 + 1     z2      0      1      1]
[     0      0      0      0      0      0      1      0      0      0      0      1      0      0      0      0      1 z2 + 1      0     z2      0 z2 + 1      0      1      0      1 z2 + 1 z2 + 1      0      0]
[     0      0      0      0      0      0      0      1      0      0      0      1      0      0      0      0      1      0 z2 + 1     z2      1      0 z2 + 1     z2      0      1      0      0     z2      1]
[     0      0      0      0      0      0      0      0      1      0      0 z2 + 1      0      0      0      0 z2 + 1 z2 + 1      1      1      1 z2 + 1     z2 z2 + 1      1      1      1 z2 + 1 z2 + 1 z2 + 1]
[     0      0      0      0      0      0      0      0      0      1      0 z2 + 1      0      0      0      0 z2 + 1      0 z2 + 1     z2 z2 + 1      0      1      1 z2 + 1     z2      1     z2     z2      1]
[     0      0      0      0      0      0      0      0      0      0      1     z2      0      0      0      0      0      0 z2 + 1 z2 + 1     z2      0      0      1 z2 + 1     z2      0 z2 + 1     z2 z2 + 1]
[     0      0      0      0      0      0      0      0      0      0      0      0      1      0      0      0      1      0      0 z2 + 1      0      1 z2 + 1 z2 + 1     z2     z2      0      0      0      0]
[     0      0      0      0      0      0      0      0      0      0      0      0      0      1      0      0      0     z2      0     z2      0      0 z2 + 1      0 z2 + 1     z2     z2     z2      0      0]
[     0      0      0      0      0      0      0      0      0      0      0      0      0      0      1      0     z2      1 z2 + 1      1     z2     z2     z2     z2      1     z2 z2 + 1      0      1 z2 + 1]
[     0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      1      0     z2 z2 + 1      1     z2      1     z2     z2     z2      1      1 z2 + 1      0      1]

In [10]:
# Encoder of C
E = GoppaEncoder(C)
word = vector(F, (b, b + 1, b, 0, b + 1, 0, 1, b + 1, b, b + 1, b + 1, b, 1, 0, b + 1))
E.encode(word)

(z2, z2 + 1, z2, 0, z2 + 1, 0, 1, z2 + 1, z2, z2 + 1, z2 + 1, z2 + 1, z2, 1, 0, z2 + 1, z2, 0, z2, 1, 1, 1, 0, z2, 0, z2 + 1, z2, z2 + 1, 0, 0)

In [11]:
# Encoder of C
E = GoppaEncoder(C)

print("\nGoppa encoder:")
print(E)

print("\nWord to encode:")
word = random_word(G.nrows(), F)
print(word)

print("\nWord encoded:")
x = E.encode(word)
print("x = " + str(x))

print("\nErrors added to x:")
num_errors = floor(g.degree()/2)
e = random_error(len(x), num_errors, F)
print("e = " + str(e))

print("\nWord encoded with errors (y = x + e):")
y = x + e
print("y = " + str(y))


Goppa encoder:
Encoder for [30, 15] Goppa code

Word to encode:
(z2, z2 + 1, 0, z2, 1, 0, 1, 0, 0, 1, 0, 1, 1, z2 + 1, z2)

Word encoded:
x = (z2, z2 + 1, 0, z2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, z2 + 1, z2, z2, z2 + 1, z2, 1, 1, z2, z2, 0, z2, 1, 0, z2 + 1, 0, z2)

Errors added to x:
e = (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, z2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

Word encoded with errors (y = x + e):
y = (z2, z2 + 1, 0, z2, 1, 0, 1, 0, 0, 1, 0, 1, 1, z2 + 1, z2 + 1, z2, z2, z2 + 1, z2, 1, 1, z2, z2, 0, z2, 1, 0, z2 + 1, 0, z2)


In [12]:
# Decoder of C
D = GoppaDecoder(C)

print("\nDecode to code `y`:")
x2 = D.decode_to_code(y)
print("x = " + str(x2))

print("\nDecode to message `y`:")
word2 = D.decode_to_message(y)
print(word2)


Decode to code `y`:
x = (z2, z2 + 1, 0, z2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, z2 + 1, z2, z2, z2 + 1, z2, 1, 1, z2, z2, 0, z2, 1, 0, z2 + 1, 0, z2)

Decode to message `y`:
(z2, z2 + 1, 0, z2, 1, 0, 1, 0, 0, 1, 0, 1, 1, z2 + 1, z2)


In [13]:
x == x2

True

In [14]:
word == word2

True

#### Example 2

In [15]:
F = GF(2^2)
L = GF(2^8)
a = L.gen()
b = F.gen()
R.<x> = L[]
g = x^5 + a*x^2 + x + a^2
n = 30
defining_set = get_defining_set(n, g, L)

In [16]:
# Goppa code
C = Goppa(defining_set, g, F)

print("Goppa code:")
print(C)

print("\nParity matrix:")
H = C.get_parity_check_matrix()
show(H)
#print(latex(H))

print("\nGenerator matrix:")
G = C.get_generator_matrix()
show(G)
#print(latex(G))

Goppa code:
[30, 10] Goppa code

Parity matrix:


20 x 30 dense matrix over Finite Field in z2 of size 2^2 (use the '.str()' method to see the entries)


Generator matrix:


[     1      0      0      0      0      0      0      0      0      0     z2      1     z2      1      1      0      0     z2      1      0      1     z2     z2 z2 + 1      0      0     z2     z2 z2 + 1      0]
[     0      1      0      0      0      0      0      0      0      0      0      0      0 z2 + 1      0 z2 + 1      1      0      1     z2      0     z2      0      0      1     z2     z2      0 z2 + 1 z2 + 1]
[     0      0      1      0      0      0      0      0      0      0      1      0     z2      0 z2 + 1     z2     z2     z2      0      1 z2 + 1 z2 + 1      1 z2 + 1     z2      1      0      0      1      0]
[     0      0      0      1      0      0      0      0      0      0     z2 z2 + 1     z2      1     z2      1      0      0      1     z2     z2      0     z2      1 z2 + 1 z2 + 1     z2 z2 + 1     z2     z2]
[     0      0      0      0      1      0      0      0      0      0      0 z2 + 1 z2 + 1     z2     z2 z2 + 1      0      1      0      1      1      1      1 z2 + 1      1     z2      1 z2 + 1     z2     z2]
[     0      0      0      0      0      1      0      0      0      0      1 z2 + 1 z2 + 1      0     z2      0      1      1      1      0      0 z2 + 1 z2 + 1      0     z2      0 z2 + 1 z2 + 1      1     z2]
[     0      0      0      0      0      0      1      0      0      0      1      1      1      1      0 z2 + 1 z2 + 1      0 z2 + 1     z2     z2      0 z2 + 1      0 z2 + 1      1      1     z2      0      0]
[     0      0      0      0      0      0      0      1      0      0      1 z2 + 1      0 z2 + 1 z2 + 1      1     z2     z2     z2 z2 + 1 z2 + 1 z2 + 1 z2 + 1      1     z2     z2     z2 z2 + 1      1     z2]
[     0      0      0      0      0      0      0      0      1      0      1 z2 + 1      0 z2 + 1     z2     z2 z2 + 1     z2      0      0      0     z2      1      1      0      0      0     z2      1      1]
[     0      0      0      0      0      0      0      0      0      1 z2 + 1 z2 + 1     z2      0      1 z2 + 1     z2      1 z2 + 1      0     z2 z2 + 1     z2      1      0 z2 + 1      1 z2 + 1 z2 + 1      0]

In [17]:
# Encoder of C
E = GoppaEncoder(C)

print("\nGoppa encoder:")
print(E)

print("\nWord to encode:")
word = random_word(G.nrows(), F)
print(word)

print("\nWord encoded:")
x = E.encode(word)
print("x = " + str(x))

print("\nErrors added to x:")
num_errors = floor(g.degree()/2)
e = random_error(len(x), num_errors, F)
print("e = " + str(e))

print("\nWord encoded with errors (y = x + e):")
y = x + e
print("y = " + str(y))


Goppa encoder:
Encoder for [30, 10] Goppa code

Word to encode:
(0, 1, 0, z2 + 1, 1, z2, z2 + 1, 0, 0, z2)

Word encoded:
x = (0, 1, 0, z2 + 1, 1, z2, z2 + 1, 0, 0, z2, 1, z2, z2 + 1, 1, z2, 0, z2, 1, z2 + 1, z2 + 1, z2, z2 + 1, 0, z2, z2 + 1, 0, z2, 0, z2 + 1, z2 + 1)

Errors added to x:
e = (z2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0)

Word encoded with errors (y = x + e):
y = (z2, 1, 0, z2 + 1, 1, z2, z2 + 1, 0, 0, z2, 1, z2, z2 + 1, 1, z2, 0, z2, 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, 0, z2, z2 + 1, 0, z2, 0, z2 + 1, z2 + 1)


In [18]:
# Decoder of C
D = GoppaDecoder(C)

print("\nDecode to code `y`:")
x2 = D.decode_to_code(y)
print("x = " + str(x2))

print("\nDecode to message `y`:")
word2 = D.decode_to_message(y)
print(word2)


Decode to code `y`:
x = (0, 1, 0, z2 + 1, 1, z2, z2 + 1, 0, 0, z2, 1, z2, z2 + 1, 1, z2, 0, z2, 1, z2 + 1, z2 + 1, z2, z2 + 1, 0, z2, z2 + 1, 0, z2, 0, z2 + 1, z2 + 1)

Decode to message `y`:
(0, 1, 0, z2 + 1, 1, z2, z2 + 1, 0, 0, z2)


In [19]:
x == x2

True

In [20]:
word == word2

True

 #### Example 3

In [21]:
F = GF(3)
L = GF(3^6)
a = L.gen()
b = F.gen()
R.<x> = L[]      
g = x^4 + a*x + 1 + a^2
n = 35
defining_set = get_defining_set(n, g, L)

In [22]:
# Goppa code
C = Goppa(defining_set, g, F)

print("Goppa code:")
print(C)

print("\nParity matrix:")
H = C.get_parity_check_matrix()
show(H)
#print(latex(H))

print("\nGenerator matrix:")
G = C.get_generator_matrix()
show(G)
#print(latex(G))

Goppa code:
[35, 11] Goppa code

Parity matrix:


24 x 35 dense matrix over Finite Field of size 3 (use the '.str()' method to see the entries)


Generator matrix:


[1 0 0 0 0 0 0 0 0 0 0 0 2 2 0 2 2 2 1 0 1 1 0 1 2 2 2 2 0 2 1 1 1 1 2]
[0 1 0 0 0 0 0 0 0 0 0 1 2 1 2 2 1 2 0 0 1 1 0 1 0 1 1 0 0 1 0 1 1 0 1]
[0 0 1 0 0 0 0 0 0 0 0 1 1 0 2 2 0 2 2 2 0 2 2 0 0 2 0 1 2 0 1 0 1 0 2]
[0 0 0 1 0 0 0 0 0 0 0 1 2 2 1 1 2 2 2 0 1 0 1 0 0 2 1 2 2 2 2 0 2 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 2 2 1 0 1 0 0 0 0 2 1 0 0 1 2 2 1 2]
[0 0 0 0 0 1 0 0 0 0 0 1 0 2 1 2 2 2 0 2 1 0 2 1 1 2 0 0 2 1 1 1 2 1 1]
[0 0 0 0 0 0 1 0 0 0 0 0 1 2 1 2 1 1 2 1 1 1 2 2 0 1 2 1 1 0 1 2 2 0 2]
[0 0 0 0 0 0 0 1 0 0 0 0 1 1 2 2 1 0 0 1 0 1 2 0 1 1 0 1 2 1 2 0 2 2 0]
[0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 2 2 1 1 0 1 2 1 2 2 0 0 1 0 0 2 2 1]
[0 0 0 0 0 0 0 0 0 1 0 2 1 0 0 1 1 0 2 0 2 0 0 1 1 0 2 1 2 1 1 1 2 2 1]
[0 0 0 0 0 0 0 0 0 0 1 1 0 2 1 1 0 1 1 2 0 2 1 0 0 0 2 1 1 1 2 2 2 1 1]

In [23]:
# Encoder of C
E = GoppaEncoder(C)

print("\nGoppa encoder:")
print(E)

print("\nWord to encode:")
word = random_word(G.nrows(), F)
print(word)

print("\nWord encoded:")
x = E.encode(word)
print("x = " + str(x))

print("\nErrors added to x:")
num_errors = floor(g.degree()/2)
e = random_error(len(x), num_errors, F)
print("e = " + str(e))

print("\nWord encoded with errors (y = x + e):")
y = x + e
print("y = " + str(y))


Goppa encoder:
Encoder for [35, 11] Goppa code

Word to encode:
(0, 0, 0, 1, 2, 1, 0, 0, 0, 1, 1)

Word encoded:
x = (0, 0, 0, 1, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 2, 1, 0, 0, 1, 2, 2, 2, 0, 1, 1)

Errors added to x:
e = (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)

Word encoded with errors (y = x + e):
y = (0, 0, 0, 1, 2, 1, 1, 0, 0, 1, 1, 2, 0, 2, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 2, 1, 0, 0, 2, 2, 2, 2, 0, 1, 1)


In [24]:
# Decoder of C
D = GoppaDecoder(C)

print("\nDecode to code `y`:")
x2 = D.decode_to_code(y)
print("x = " + str(x2))

print("\nDecode to message `y`:")
word2 = D.decode_to_message(y)
print(word2)


Decode to code `y`:
x = (0, 0, 0, 1, 2, 1, 2, 0, 0, 1, 1, 2, 0, 2, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 2, 1, 0, 0, 0, 2, 2, 2, 0, 1, 1)

Decode to message `y`:


ValueError: matrix equation has no solutions

In [ ]:
x == x2

In [ ]:
word == word2

#### Example 4

In [ ]:
F = GF(2)
L = GF(2^6)
a = L.gen()
b = F.gen()
R.<x> = L[]
g = x^5 + a*x^3
n = 30
defining_set = get_defining_set(n, g, L)

In [ ]:
# Goppa code
C = Goppa(defining_set, g, F)

print("Goppa code:")
print(C)

print("\nParity matrix:")
H = C.get_parity_check_matrix()
show(H)
#print(latex(H))

print("\nGenerator matrix:")
G = C.get_generator_matrix()
show(G)
#print(latex(G))

In [ ]:
# Encoder of C
E = GoppaEncoder(C)

print("\nGoppa encoder:")
print(E)

print("\nWord to encode:")
word = random_word(G.nrows(), F)
print(word)

print("\nWord encoded:")
x = E.encode(word)
print("x = " + str(x))

print("\nErrors added to x:")
num_errors = floor(g.degree()/2)
e = random_error(len(x), num_errors, F)
print("e = " + str(e))

print("\nWord encoded with errors (y = x + e):")
y = x + e
print("y = " + str(y))

In [ ]:
# Decoder of C
D = GoppaDecoder(C)

print("\nDecode to code `y`:")
x2 = D.decode_to_code(y)
print("x = " + str(x2))

print("\nDecode to message `y`:")
word2 = D.decode_to_message(y)
print(word2)

In [ ]:
x == x2

In [ ]:
word == word2

# **ALGORITMO DE SUGIYAMA**

1. Calcular el síndrome $S(x)$.
2. Sean $r_{-1}(x) = g(x)$, $r_0(x) = S(x)$, $U_{-1}(x) = 0$ y $U_0(x) = 1$.
3. Buscar $q_i(x)$ y $r_i(x)$ aplicando el algoritmo de Euclides para encontrar el máximo común divisor de $r_{i-2}(x)$ y $r_{i-1}(x)$ para $i = 1,..., k$, hasta que $k$ cumpla que $gr(r_{k-1}(x)) \geq t$ y $gr(r_k(x)) < t$:

$$r_{i-2}(x) = r_{i-1}(x) q_i(x) + r_i(x), \qquad gr(r_i(x)) < gr(r_{i-1})(x)$$
    
4. Calcular $U_k(x)$, donde
    
$$U_i(x) = q_i(x) U_{i-1}(x) + U_{i-2}(x)$$

5. La solución viene dada por:
$$\eta(x) = (-1)^k \delta r_k(x)$$
$$\sigma(x) = \delta U_k(x)$$

In [ ]:
F = GF(2^2)
L = GF(2^4)
a = L.gen()
b = F.gen()
R.<x> = L[]
g = x^3 + a*x^2 + 1
n = 10
defining_set = get_defining_set(n, g, L)
C = Goppa(defining_set, g, F)
G = C.get_generator_matrix()
E = GoppaEncoder(C)
D = GoppaDecoder(C)
word = random_word(G.nrows(), F)
print(word)
x = E.encode(word)
print(x)
num_errors = floor(g.degree()/2)
e = random_error(len(x), num_errors, F)
y = x + e
D.decode_to_message(y)

In [ ]:
D.get_syndrome(word)

In [34]:
F = GF(3^3)
L = GF(3^6)
a = L.gen()
b = F.gen()
R.<x> = L[]
g = x^2 + a*x + 1 + a^2
n = 10
defining_set = get_defining_set(n, g, L)
C = Goppa(defining_set, g, F)
print(C)
print(C.get_generator_matrix().nrows())
E = GoppaEncoder(C)
word = vector(F, (1, 0, b + 1, b, 0, b + 1))
x = E.encode(word)
print(x)

[10, 6] Goppa code
6
(1, 0, z3 + 1, z3, 0, z3 + 1, 2*z3, z3^2, z3 + 1, 0)
